In [1]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Conv2DTranspose, \
    UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import Adam
import numpy as np

2021-12-07 21:38:52.105163: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
vgg19 = VGG19(input_shape=(224,224,3)) # include_top=True, weights='imagenet'
vgg19.trainable = False
x = Conv2D(64, (3,3), padding='same', activation='relu')(vgg19.layers[-16].output)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = MaxPool2D((2,2))(x)
x = Conv2DTranspose(64, (3,3), padding='same', activation='relu')(x)
x = UpSampling2D((2,2))(x)
x = Conv2DTranspose(64, (3,3), padding='same', activation='relu')(x)
x = UpSampling2D((2,2))(x)
x = Conv2DTranspose(64, (3,3), padding='same', activation='relu')(x)
x = UpSampling2D((2,2))(x)
o1 = Conv2D(3, (3,3), padding='same')(x)
o2 = Conv2D(3, (3,3), padding='same')(o1)

model = Model(vgg19.input, [o1,o2])
model.summary()

2021-12-07 21:38:55.997015: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-12-07 21:38:57.189381: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 21:38:57.191416: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1650 with Max-Q Design computeCapability: 7.5
coreClock: 1.125GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 104.34GiB/s
2021-12-07 21:38:57.191566: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-12-07 21:38:57.212176: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-12-07 21:38:57.212541: I tensorflow/s

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
model.compile(loss=["mse","mse"], optimizer=Adam(learning_rate=1e-4), \
    loss_weights=[1,2])

In [4]:
dataX = np.random.random((2,224,224,3))
dataY = np.random.random((2,224,224,3))

In [5]:
model.fit(dataX, dataY, epochs=1)

2021-12-07 21:39:13.210460: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-07 21:39:13.228271: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1599960000 Hz
2021-12-07 21:39:13.839935: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-12-07 21:39:14.279775: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2021-12-07 21:39:14.877607: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-12-07 21:39:15.219213: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-12-07 21:39:15.361575: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.09GiB with freed_by_count=0. The caller indicat

1/1 [==============================] - 4s 4s/step - loss: 160.9800 - conv2d_2_loss: 62.7244 - conv2d_3_loss: 49.1278
